<a href="https://colab.research.google.com/github/catnipglitch/google-colab-notebooks-catnip/blob/main/openai/OpenAI_ImageEdit_Example001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI Image Edit API を使用した画像編集ツール

このノートブックは、OpenAIのImage Edit APIを使用して画像を編集するためのインタラクティブなツールを提供します。

## 機能

* **画像のアップロード:** オリジナル画像とマスク画像（オプション）をアップロードできます。
* **プロンプト入力:** 画像編集のためのプロンプトを入力できます。
* **画像サイズ、品質、背景の選択:** 生成される画像のサイズ、品質、背景を指定できます。
* **画像編集と保存:** アップロードした画像とプロンプトに基づいて画像を編集し、Google Driveに保存できます。
* **APIレスポンスの保存:** APIからのレスポンスはJSONファイルとしてGoogle Driveに保存されます。

## 使用方法

1. OpenAI APIキーを `userdata` に設定します。
2. オリジナル画像とマスク画像（オプション）をアップロードします。
3. 画像編集のためのプロンプトを入力します。
4. 画像サイズ、品質、背景を選択します。
5. "画像を編集して保存" ボタンをクリックして、画像を編集し、Google Driveに保存します。

## 注意

* このノートブックは、OpenAIのImage Edit APIを使用するために、`openai` ライブラリがインストールされている必要があります。
* Google Driveにアクセスするために、Google Colabの認証が必要です。

In [ ]:
#初期化
!pip install Pillow
!pip install openai


import ipywidgets as widgets
from IPython.display import display, Image # displayは重複しているので1つに
from google.colab import files, userdata, drive # google.colabからのimportをまとめる
from PIL import Image
import io
import os
import base64
import json
import datetime
from pathlib import Path
from openai import OpenAI
import tempfile


# 一時ディレクトリを作成
temp_dir = tempfile.mkdtemp()

# Googleドライブを一度だけマウントするのだ！
drive.mount('/content/drive')

# 保存先のディレクトリを設定するのだ
SAVE_DIR = '/content/drive/MyDrive/AI_Generated_Images'
os.makedirs(SAVE_DIR, exist_ok=True)

# API接続の準備をするのだ
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

# ファイル名を生成する関数を定義するのだ
def generate_unique_filename() -> str:
    """重複しないタイムスタンプベースのファイル名を生成するのだ"""
    timestamp = datetime.datetime.now().strftime('%Y-%m%d-%H%M-%S')
    base_filename = timestamp

    index = 0
    while True:
        filename = base_filename if index == 0 else f"{base_filename}_{index}"
        img_path = os.path.join(SAVE_DIR, f"{filename}.png")
        json_path = os.path.join(SAVE_DIR, f"{filename}.json")

        if not os.path.exists(img_path) and not os.path.exists(json_path):
            return filename

        index += 1


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 画像生成プロンプトまとめ
prompt = "画風をジブリ風に変更したイラストを出力してください。" # @param {type:"string"}
img_size = '1024x1024' # @param ["1024x1024", "1536x1024", "1024x1536"]
img_quality = 'high' # @param ["high", "medium","low"]
img_background = 'opaque' # @param ["transparent", "opaque"]


In [ ]:
# ## スタイルの初期化

# %%
# ボタンの幅を指定 (例: 800px)
button_layout = widgets.Layout(width='400px')
# FileUploadウィジェットのスタイルを設定
style = {'description_width': 'initial', 'width': '400px'}  # 幅を800pxに設定


# ファイルアップロードウィジェット
original_uploader = widgets.FileUpload(accept='image/*', multiple=False, description="オリジナルファイル", style=style, layout=button_layout)
mask_uploader = widgets.FileUpload(accept='image/*', multiple=False, description="マスクファイル（オプション）", style=style, layout=button_layout)

# クリアボタン
clear_original_button = widgets.Button(description="オリジナルファイルのクリア", layout=button_layout)
clear_mask_button = widgets.Button(description="マスクファイルのクリア", layout=button_layout)
clear_both_button = widgets.Button(description="両方のファイルのクリア", layout=button_layout)

# 画像表示エリア
original_image_output = widgets.Output()
mask_image_output = widgets.Output()
image_display = widgets.HBox([original_image_output, mask_image_output])

# フォーム
form = widgets.VBox([
    original_uploader,
    mask_uploader,
    clear_original_button,
    clear_mask_button,
    clear_both_button,
    image_display
])
display(form)
# 編集ボタンを追加
# edit_button = widgets.Button(description="画像を編集して保存", layout=button_layout) # Indentation removed
form.children += (edit_button,) # formにedit_buttonを追加

# ボタンにイベントハンドラを登録
edit_button.on_click(on_edit_button_clicked)


# ファイル保存と表示の関数
def display_image(uploader, output_widget):
    if uploader.value:
        uploaded_filename = list(uploader.value.keys())[0]
        uploaded_file = uploader.value[uploaded_filename]

        # 一時ファイルに保存
        temp_filepath = os.path.join(temp_dir, uploaded_filename)
        with open(temp_filepath, 'wb') as f:
            f.write(uploaded_file['content'])

        # 画像を表示
        with output_widget:
            output_widget.clear_output()
            image = Image.open(io.BytesIO(uploaded_file['content']))

            # 画像サイズを制限
            max_width = 400  # 最大幅を400pxに設定 (フィールドの50%程度を想定)
            width_percent = (max_width / float(image.size[0]))
            height_size = int((float(image.size[1]) * float(width_percent)))
            image = image.resize((max_width, height_size))

            display(image)
        return temp_filepath
    else:
        return None

# クリアボタンのクリックイベント
def clear_uploader(uploader):
    uploader.value.clear()
    uploader._counter = 0

def on_clear_original_button_clicked(b):
    clear_uploader(original_uploader)
    with original_image_output:
        original_image_output.clear_output()

def on_clear_mask_button_clicked(b):
    clear_uploader(mask_uploader)
    with mask_image_output:
        mask_image_output.clear_output()

def on_clear_both_button_clicked(b):
    on_clear_original_button_clicked(b)
    on_clear_mask_button_clicked(b)


# ボタンにイベントハンドラを登録
clear_original_button.on_click(on_clear_original_button_clicked)
clear_mask_button.on_click(on_clear_mask_button_clicked)
clear_both_button.on_click(on_clear_both_button_clicked)

# ファイルアップロード時のイベントハンドラ (変更)
def on_original_upload_change(change):
    original_filepath = display_image(original_uploader, original_image_output)
    # ここで original_filepath を生成AIのAPIに渡す処理などを記述
    # (変更なし)

def on_mask_upload_change(change):
    mask_filepath = display_image(mask_uploader, mask_image_output)
    # ここで mask_filepath を生成AIのAPIに渡す処理などを記述
    # (変更なし)

original_uploader.observe(on_original_upload_change, names='value')
mask_uploader.observe(on_mask_upload_change, names='value')

print(f"一時ファイルは {temp_dir} に保存されます。")

一時ファイルは /tmp/tmpladga3nh に保存されます。
画像とレスポンスは 2025-0506-1447-26.png と 2025-0506-1447-26.json として Google Drive に保存されました。


In [ ]:

# 画像編集と保存の関数
def edit_and_save_image(original_filepath, mask_filepath, prompt, img_size, img_quality, img_background):
    """Open AI Create Image Edit API を使用して画像を編集し、Google Drive に保存する"""
    # リクエスト内容を表示
    input_files = [original_filepath]
    if mask_filepath:
        input_files.append(mask_filepath)
    print(f"リクエスト中... 入力ファイル: {input_files}")

    try:
        # 画像ファイルを開く
        image_file = open(original_filepath, "rb")
        # mask_file = open(mask_filepath, "rb") if mask_filepath else None # マスクファイルがある場合のみ開く # 修正: kwargsで処理

        # Open AI Create Image Edit API を呼び出す
        # モデルがgpt-image-1の場合のみbackgroundパラメータを渡す # 修正: kwargsで処理
        kwargs = {
            "image": image_file,
            "prompt": prompt,
            "model": "gpt-image-1", # モデルを指定
            "size": img_size,
            "quality": img_quality,
            "n": 1, # 画像生成数を1に設定
        }

        # マスクファイルがある場合のみ、kwargsにmaskを追加 # 修正: 条件を追加
        if mask_filepath:
            kwargs["mask"] = open(mask_filepath, "rb")

        # if kwargs["model"] == "gpt-image-1": # gpt-image-1専用のパラメータなのでコメントアウト # 修正: 不要になった
            # kwargs["background"] = img_background  # background parameter only for gpt-image-1 # 修正: 不要になった

        result = client.images.edit(**kwargs)

        # 結果から画像データを取得
        image_base64 = result.data[0].b64_json
        image_bytes = base64.b64decode(image_base64)

        # ファイル名を生成
        filename = generate_unique_filename()

        # 画像を保存
        img_path = os.path.join(SAVE_DIR, f"{filename}.png")
        with open(img_path, "wb") as f:
            f.write(image_bytes)

        # APIレスポンスをJSONファイルとして保存
        response_data = {
            "created": result.created,
            "prompt": prompt,  # ユーザーのプロンプト
            "model": "gpt-image-1",  # 使用したモデル
            "size": img_size,  # 画像サイズ
            "quality": img_quality,  # 画像品質
            # "background": img_background, # background parameter is deprecated # background パラメータは廃止されました
            "usage": result.usage.__dict__ # Usageオブジェクトを辞書に変換
        }
        # input_tokens_details を辞書に変換
        response_data["usage"]["input_tokens_details"] = response_data["usage"]["input_tokens_details"].__dict__

        json_path = os.path.join(SAVE_DIR, f"{filename}.json")
        with open(json_path, "w") as f:
            json.dump(response_data, f, indent=2)

        print(f"画像とレスポンスは {filename}.png と {filename}.json として Google Drive に保存されました。")

    except Exception as e:
        print(f"エラーが発生しました: {e}")

    finally:
        # ファイルを閉じる
        image_file.close()
        # if mask_file: # 修正: kwargsで処理したので不要になった
        #     mask_file.close()
        if "mask" in kwargs and kwargs["mask"]: # 修正: maskがkwargsにあり、かつNoneでない場合に閉じる
            kwargs["mask"].close()



# ファイルアップロード時のイベントハンドラ (変更)
def on_original_upload_change(change):
    original_filepath = display_image(original_uploader, original_image_output)
    # ここで original_filepath を生成AIのAPIに渡す処理などを記述

def on_mask_upload_change(change):
    mask_filepath = display_image(mask_uploader, mask_image_output)
    # ここで mask_filepath を生成AIのAPIに渡す処理などを記述

# ボタンクリック時のイベントハンドラ (新規追加)
def on_edit_button_clicked(b):
    original_filepath = display_image(original_uploader, original_image_output)  # 修正
    mask_filepath = display_image(mask_uploader, mask_image_output)  # 修正

    if original_filepath: # and mask_filepath:  # original_filepath and mask_filepath がNoneでないことを確認
        edit_and_save_image(original_filepath, mask_filepath, prompt, img_size, img_quality, img_background)
    else:
        print("オリジナルファイルまたはマスクファイルがアップロードされていません。") # メッセージを修正

# 編集ボタンを追加
edit_button = widgets.Button(description="画像を編集して保存", layout=button_layout)
form.children += (edit_button,) # formにedit_buttonを追加

# ボタンにイベントハンドラを登録
edit_button.on_click(on_edit_button_clicked)



# Gemini向け資料
## OpenAI Image Edit API - Request Body

このエンドポイントは、1つ以上のソース画像とプロンプトに基づいて、編集または拡張された画像を作成します。gpt-image-1とdall-e-2のみをサポートしています。

### パラメータ

| パラメータ | タイプ | 説明 | 必須 |
|---|---|---|---|
| `image` | string or array | 編集する画像。サポートされている画像ファイルまたは画像の配列である必要があります。gpt-image-1の場合、各画像は25MB未満のpng、webp、またはjpgファイルである必要があります。最大16枚の画像を提供できます。dall-e-2の場合、提供できる画像は1枚だけで、4MB未満の正方形のpngファイルである必要があります。 | **必須** |
| `prompt` | string | 目的の画像のテキストによる説明。dall-e-2の最大長は1000文字、gpt-image-1の最大長は32000文字です。 | **必須** |
| `background` | string or null | 生成される画像の背景の透明度を設定できます。このパラメータは、gpt-image-1でのみサポートされています。transparent、opaque、またはauto（デフォルト値）のいずれかである必要があります。autoを使用すると、モデルは画像に最適な背景を自動的に決定します。transparentの場合、出力形式は透明度をサポートする必要があるため、png（デフォルト値）またはwebpに設定する必要があります。 | オプション |
| `mask` | file | imageが編集される場所を示す、完全に透明な領域（アルファがゼロの場所など）を持つ追加の画像。複数の画像が提供されている場合、マスクは最初の画像に適用されます。有効なPNGファイルで、4MB未満で、imageと同じ寸法である必要があります。 | オプション |
| `model` | string | 画像生成に使用するモデル。dall-e-2とgpt-image-1のみがサポートされています。gpt-image-1に固有のパラメータが使用されない限り、デフォルトはdall-e-2です。 | オプション |
| `n` | integer or null | 生成する画像の数。1から10の間である必要があります。 | オプション |
| `quality` | string or null | 生成される画像の品質。high、medium、lowはgpt-image-1でのみサポートされています。dall-e-2は標準品質のみをサポートしています。デフォルトはautoです。 | オプション |
| `response_format` | string or null | 生成された画像が返される形式。urlまたはb64_jsonのいずれかである必要があります。URLは、画像が生成されてから60分間のみ有効です。gpt-image-1は常にbase64エンコードされた画像を返すため、このパラメータはdall-e-2でのみサポートされています。 | オプション |
| `size` | string or null | 生成される画像のサイズ。gpt-image-1の場合は1024x1024、1536x1024（横長）、1024x1536（縦長）、またはauto（デフォルト値）のいずれか、dall-e-2の場合は256x256、512x512、または1024x1024のいずれかである必要があります。 | オプション |
| `user` | string | エンドユーザーを表す一意の識別子。OpenAIが不正使用を監視および検出するのに役立ちます。 | オプション |


### 戻り値

画像オブジェクトのリストを返します。

Open AI Create Image Edit Example

# Example request
```
# これはコードとして書式設定されます
import base64
from openai import OpenAI
client = OpenAI()

prompt = """
Generate a photorealistic image of a gift basket on a white background
labeled 'Relax & Unwind' with a ribbon and handwriting-like font,
containing all the items in the reference pictures.
"""

result = client.images.edit(
    model="gpt-image-1",
    image=[
        open("body-lotion.png", "rb"),
        open("bath-bomb.png", "rb"),
        open("incense-kit.png", "rb"),
        open("soap.png", "rb"),
    ],
    prompt=prompt
)

image_base64 = result.data[0].b64_json
image_bytes = base64.b64decode(image_base64)

# Save the image to a file
with open("gift-basket.png", "wb") as f:
    f.write(image_bytes)

```

# Response
```
{
  "created": 1713833628,
  "data": [
    {
      "b64_json": "..."
    }
  ],
  "usage": {
    "total_tokens": 100,
    "input_tokens": 50,
    "output_tokens": 50,
    "input_tokens_details": {
      "text_tokens": 10,
      "image_tokens": 40
    }
  }
}
```
